In [1]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Get the variables from the environment
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

In [2]:
# Connect to PostgreSQL database
try:
    connection = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password,
        port=db_port
    )
    
    # Create a cursor object
    cursor = connection.cursor()

    # Execute SQL queries to get data from all three tables
    query_alcohol_consumption_per_capita = "SELECT * FROM alcohol_consumption_per_capita"  # Query to fetch data from alcohol_consumption_per_capita table
    query_alcohol_consumption_vs_gdp = "SELECT * FROM alcohol_consumption_vs_gdp"  # Query to fetch data from alcohol_consumption_vs_gdp table
    query_alcohol_related_mortality = "SELECT * FROM alcohol_related_mortality"  # Query to fetch data from alcohol_related_mortality table

    # Fetch alcohol_consumption_per_capita data
    cursor.execute(query_alcohol_consumption_per_capita)
    data_alcohol_consumption_per_capita = cursor.fetchall()
    col_names_alcohol_consumption_per_capita = [desc[0] for desc in cursor.description]

    # Fetch alcohol_consumption_vs_gdp data
    cursor.execute(query_alcohol_consumption_vs_gdp)
    data_alcohol_consumption_vs_gdp = cursor.fetchall()
    col_names_alcohol_consumption_vs_gdp = [desc[0] for desc in cursor.description]

    # Fetch alcohol_related_mortality data
    cursor.execute(query_alcohol_related_mortality)
    data_alcohol_related_mortality = cursor.fetchall()
    col_names_alcohol_related_mortality = [desc[0] for desc in cursor.description]

    # Create Pandas DataFrames from the fetched data
    df_alcohol_consumption_per_capita = pd.DataFrame(data_alcohol_consumption_per_capita, columns=col_names_alcohol_consumption_per_capita)
    df_alcohol_consumption_vs_gdp = pd.DataFrame(data_alcohol_consumption_vs_gdp, columns=col_names_alcohol_consumption_vs_gdp)
    df_alcohol_related_mortality = pd.DataFrame(data_alcohol_related_mortality, columns=col_names_alcohol_related_mortality)
    
    # Merge DataFrames on common columns ('entity', and 'year')
    df_merged_1 = pd.merge(df_alcohol_consumption_per_capita, df_alcohol_consumption_vs_gdp, on=['entity', 'year', 'alcohol_consumption_per_capita'], how='outer')
    df_global_alcohol = pd.merge(df_merged_1, df_alcohol_related_mortality, on=['entity', 'year'], how='outer')
    
    # Display the merged DataFrame
    print(df_global_alcohol)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()


           entity  year alcohol_consumption_per_capita  \
0     Afghanistan  2000                        0.00277   
1     Afghanistan  2002                           None   
2     Afghanistan  2003                           None   
3     Afghanistan  2004                           None   
4     Afghanistan  2005                        0.02684   
...           ...   ...                            ...   
6608     Zimbabwe  2018                           None   
6609     Zimbabwe  2019                        3.62809   
6610     Zimbabwe  2020                           None   
6611     Zimbabwe  2021                           None   
6612     Zimbabwe  2022                           None   

     alcohol_consumption_vs_gdp alcohol_related_mortality  
0                          None                       NaN  
1                     1280.4631                       NaN  
2                     1292.3335                       NaN  
3                     1260.0605                       NaN  
4  

In [3]:
# Example: Show the first 5 rows
df_global_alcohol.head()

,entity,year,alcohol_consumption_per_capita,alcohol_consumption_vs_gdp,alcohol_related_mortality
0,Afghanistan,2000,0.00277,None,NaN
1,Afghanistan,2002,None,1280.4631,NaN
2,Afghanistan,2003,None,1292.3335,NaN
3,Afghanistan,2004,None,1260.0605,NaN
4,Afghanistan,2005,0.02684,1352.3207,NaN
